In [1]:
import torch

In [2]:
model = torch.hub.load('facebookresearch/detr:main', 'detr_resnet50', pretrained=True)

Downloading: "https://github.com/facebookresearch/detr/archive/main.zip" to C:\Users\Administrator/.cache\torch\hub\main.zip
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Administrator/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:08<00:00, 12.0MB/s]
Downloading: "https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth" to C:\Users\Administrator/.cache\torch\hub\checkpoints\detr-r50-e632da11.pth
100%|██████████| 159M/159M [00:14<00:00, 11.5MB/s] 


In [3]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_set_aux_loss',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_m

In [4]:
from PIL import Image 

img = Image.open('zidane.jpg')

In [8]:
import torchvision.transforms as T
model.eval()
img_tensor = T.ToTensor()(img)
outputs = model(img_tensor.unsqueeze(0))
print(outputs)

C:\Users\Administrator/.cache\torch\hub\facebookresearch_detr_main\models\position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


{'pred_logits': tensor([[[-17.8277,  -1.1183,  -6.8325,  ...,  -8.7809,  -4.2013,   9.7501],
         [-18.9028,  -1.2861, -10.1813,  ..., -12.6598,  -2.5825,  11.8032],
         [-20.4857,   1.2731,  -9.8734,  ..., -11.6334,  -7.3509,  11.7145],
         ...,
         [-16.6541,  -1.2568, -10.0348,  ...,  -8.2975,  -2.4265,   9.9010],
         [-14.9384,  10.5533,  -6.4617,  ..., -11.6251,  -5.8903,   6.7458],
         [-15.8555,  -2.2162,  -8.9177,  ...,  -9.2907,  -2.2909,  10.1224]]],
       grad_fn=<SelectBackward0>), 'pred_boxes': tensor([[[0.2229, 0.8131, 0.1206, 0.3573],
         [0.5104, 0.5517, 0.0200, 0.0575],
         [0.2801, 0.7133, 0.3368, 0.5556],
         [0.7671, 0.4661, 0.0600, 0.2114],
         [0.5825, 0.6371, 0.6216, 0.7030],
         [0.6112, 0.6368, 0.6080, 0.7101],
         [0.4888, 0.6152, 0.0345, 0.1509],
         [0.3671, 0.8038, 0.0551, 0.3890],
         [0.3593, 0.7353, 0.0386, 0.3263],
         [0.5168, 0.4418, 0.7841, 0.4103],
         [0.8012, 0.5830, 0

def 

In [10]:
outputs['pred_logits'].shape
# 进行100个框的预测，每个框有91个类别的预测值，每个类别的预测值是一个0-1的概率值，还有一个背景类别

torch.Size([1, 100, 92])

In [ ]:
from torch import nn
def pred_logits_loss(pred_logits, target):
    """
    pred_logits: (N, num_queries, num_classes) -> (N, 100, 91)
    target: (N, num_queries) -> (N, 100)
    """
    # 交叉熵损失函数
    criterion = nn.CrossEntropyLoss()
    # 损失函数的输入是 (N, num_queries, num_classes) 和 (N, num_queries)
    # 损失函数的输出是 (N, num_queries)
    loss = criterion(pred_logits, target)
    return loss

In [ ]:
def pred_boxes_loss(pred_boxes, target):
    """
    pred_boxes: (N, num_queries, 4) -> (N, 100, 4)
    target: (N, num_queries, 4) -> (N, 100, 4)
    """
    # 均方损失函数
    criterion = nn.MSELoss()
    # 损失函数的输入是 (N, num_queries, 4) 和 (N, num_queries, 4)
    # 损失函数的输出是 (N, num_queries)
    loss = criterion(pred_boxes, target)
    return loss

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
from torch import utils
import math
import sys

# train
def train_one_epoch(model, criterion, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger